In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore")

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

from datetime import datetime
import datetime as dt

## 1. 데이터 전처리

#### 교통사고수 데이터

In [3]:
###### 교통사고 데이터 불러오기
data = pd.read_excel("data/2014_19년 서울_경기 개별 교통사고 정보.xlsx")
data.head()

,발생일,발생시간,사망자수,중상자수,경상자수,부상신고자수,발생지_시도,발생지_시군구,법정동명,사고유형_대분류,...,노면상태,기상상태,가해당사자종별,가해자성별,가해자연령,가해자신체상해정도,피해당사자종별,피해자성별,피해자연령,피해자신체상해정도
0,20140101,00시,0,0,1,0,서울,종로구,숭인동,차대차,...,젖음/습기,흐림,승합차,남,48세,기타불명,승용차,남,65세,상해없음
1,20140101,00시,0,0,1,0,서울,강북구,번동,차대사람,...,건조,맑음,불명,기타불명,불명,기타불명,보행자,남,60세,경상
2,20140101,00시,0,1,0,0,서울,강동구,천호동,차대사람,...,건조,맑음,승용차,남,38세,상해없음,보행자,남,59세,중상
3,20140101,00시,0,0,2,0,서울,구로구,궁동,차대차,...,건조,맑음,승용차,남,25세,상해없음,승용차,남,41세,경상
4,20140101,00시,0,0,0,2,서울,구로구,오류동,차대차,...,건조,맑음,승용차,남,31세,상해없음,승용차,남,65세,부상신고


In [4]:
# 발생일은 2018년~2019년
ta = data[data["발생일"] >= 20180101]

# 발생지_시도 에서 경기도 데이터 삭제
ta = ta[ta["발생지_시도"] == "서울"]

ta.head()
ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78053 entries, 363827 to 550787
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   발생일        78053 non-null  int64 
 1   발생시간       78053 non-null  object
 2   사망자수       78053 non-null  int64 
 3   중상자수       78053 non-null  int64 
 4   경상자수       78053 non-null  int64 
 5   부상신고자수     78053 non-null  int64 
 6   발생지_시도     78053 non-null  object
 7   발생지_시군구    78053 non-null  object
 8   법정동명       78052 non-null  object
 9   사고유형_대분류   78053 non-null  object
 10  사고유형_중분류   78053 non-null  object
 11  사고유형       78053 non-null  object
 12  가해자법규위반    78053 non-null  object
 13  도로형태_대분류   78053 non-null  object
 14  도로형태       78053 non-null  object
 15  노면상태_대분류   78053 non-null  object
 16  노면상태       78053 non-null  object
 17  기상상태       78053 non-null  object
 18  가해당사자종별    78053 non-null  object
 19  가해자성별      78053 non-null  object
 20  가해자연령      78053 non-n

In [5]:
# 발생일, 사망자수, 중상자수, 경상자수, 발생지_시군구 열만 남기기
ta = ta.loc[:,['발생일','사망자수','중상자수','경상자수','발생지_시군구']]
ta.head()

,발생일,사망자수,중상자수,경상자수,발생지_시군구
363827,20180101,0,0,2,용산구
363828,20180101,0,1,1,강동구
363829,20180101,0,0,1,송파구
363830,20180101,0,1,0,성북구
363835,20180101,0,0,1,강북구


In [6]:
##### 발생일 데이터 타입은 datetime으로 변경
ta["발생일"] = ta["발생일"].astype(str)
ta['발생일'] = pd.to_datetime(ta["발생일"]).dt.normalize()

In [7]:
ta.head()

,발생일,사망자수,중상자수,경상자수,발생지_시군구
363827,2018-01-01,0,0,2,용산구
363828,2018-01-01,0,1,1,강동구
363829,2018-01-01,0,0,1,송파구
363830,2018-01-01,0,1,0,성북구
363835,2018-01-01,0,0,1,강북구


#### 강우량 데이터

In [8]:
###### 강우량 데이터 불러오기 (2018~2019)
rainfall_2018 = pd.read_csv("data/강우량 데이터/2018년 서울 일별 강우량 데이터.csv")
rainfall_2019 = pd.read_csv("data/강우량 데이터/2019년 서울 일별 강우량 데이터.csv")

rainfall_2018.head()

,\t지점번호,지점명,일시,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각
0,\t400,강남,2018-01-01,0.0,0.0,0:01
1,\t400,강남,2018-01-02,0.0,0.0,0:01
2,\t400,강남,2018-01-03,0.0,0.0,0:01
3,\t400,강남,2018-01-04,0.0,0.0,0:01
4,\t400,강남,2018-01-05,0.0,0.0,0:01


In [9]:
rainfall_2019.head()

,\t지점번호,지점명,일시,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각
0,\t400,강남,2019-01-01,0.0,0.0,0:01
1,\t400,강남,2019-01-02,0.0,0.0,0:01
2,\t400,강남,2019-01-03,0.0,0.0,0:01
3,\t400,강남,2019-01-04,0.0,0.0,0:01
4,\t400,강남,2019-01-05,0.0,0.0,0:01


In [10]:
# 2018~2019 데이터 합치기
rainfall = pd.concat([rainfall_2018, rainfall_2019])
rainfall.head()

,\t지점번호,지점명,일시,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각
0,\t400,강남,2018-01-01,0.0,0.0,0:01
1,\t400,강남,2018-01-02,0.0,0.0,0:01
2,\t400,강남,2018-01-03,0.0,0.0,0:01
3,\t400,강남,2018-01-04,0.0,0.0,0:01
4,\t400,강남,2018-01-05,0.0,0.0,0:01


In [11]:
# 현충원 -> 동작, 북악산 -> 종로
rainfall.loc[rainfall["지점명"]=="현충원","지점명"] = "동작"
rainfall.loc[rainfall["지점명"]=="북악산","지점명"] = "종로"
rainfall[rainfall["지점명"] == "동작"]
rainfall[rainfall["지점명"] == "종로"]

,\t지점번호,지점명,일시,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각
7660,\t422,종로,2018-01-01,0.0,0.0,0:01
7661,\t422,종로,2018-01-02,0.0,0.0,0:01
7662,\t422,종로,2018-01-03,0.0,0.0,0:01
7663,\t422,종로,2018-01-04,0.0,0.0,0:01
7664,\t422,종로,2018-01-05,0.0,0.0,0:01
...,...,...,...,...,...,...
7991,\t422,종로,2018-11-28,0.0,0.0,0:01
7992,\t422,종로,2018-11-29,0.0,0.0,0:01
7993,\t422,종로,2018-11-30,0.0,0.0,0:01
7994,\t422,종로,2018-12-01,0.0,0.0,0:01


In [12]:
# 지점명 = '기상청' 데이터 삭제
idx = rainfall[rainfall["지점명"]=="기상청"].index
rainfall = rainfall.drop(idx)

In [13]:
# 지점명 뒤에 '구' 붙이기
rainfall["지점명"] = rainfall["지점명"] + "구"
rainfall.head()

,\t지점번호,지점명,일시,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각
0,\t400,강남구,2018-01-01,0.0,0.0,0:01
1,\t400,강남구,2018-01-02,0.0,0.0,0:01
2,\t400,강남구,2018-01-03,0.0,0.0,0:01
3,\t400,강남구,2018-01-04,0.0,0.0,0:01
4,\t400,강남구,2018-01-05,0.0,0.0,0:01


In [14]:
# 지점번호, 1시간최다강수량시각 삭제
rainfall = rainfall.drop(columns=['\t지점번호', '1시간최다강수량시각'], axis=1)

In [15]:
rainfall.head()

,지점명,일시,강수량(mm),1시간최다강수량(mm)
0,강남구,2018-01-01,0.0,0.0
1,강남구,2018-01-02,0.0,0.0
2,강남구,2018-01-03,0.0,0.0
3,강남구,2018-01-04,0.0,0.0
4,강남구,2018-01-05,0.0,0.0


In [16]:
# 지점명 -> 발생지_시군구 , 일시 -> 발생일 이름 변경
rainfall = rainfall.rename(columns={'지점명':'발생지_시군구','일시':'발생일'})
rainfall.head()

,발생지_시군구,발생일,강수량(mm),1시간최다강수량(mm)
0,강남구,2018-01-01,0.0,0.0
1,강남구,2018-01-02,0.0,0.0
2,강남구,2018-01-03,0.0,0.0
3,강남구,2018-01-04,0.0,0.0
4,강남구,2018-01-05,0.0,0.0


In [17]:
rainfall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19307 entries, 0 to 9850
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   발생지_시군구       19295 non-null  object 
 1   발생일           19295 non-null  object 
 2   강수량(mm)       19273 non-null  float64
 3   1시간최다강수량(mm)  19277 non-null  float64
dtypes: float64(2), object(2)
memory usage: 754.2+ KB


In [18]:
# 발생일을 datetime 형식으로 변경
rainfall['발생일'] = pd.to_datetime(rainfall["발생일"]).dt.normalize()
rainfall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19307 entries, 0 to 9850
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   발생지_시군구       19295 non-null  object        
 1   발생일           19295 non-null  datetime64[ns]
 2   강수량(mm)       19273 non-null  float64       
 3   1시간최다강수량(mm)  19277 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 754.2+ KB


In [19]:
rainfall
len(rainfall)

19307

In [20]:
# 인덱스 다시 설정 
rainfall["index"] = range(0,len(rainfall))
rainfall["index"]
rainfall = rainfall.set_index("index")
rainfall

,발생지_시군구,발생일,강수량(mm),1시간최다강수량(mm)
index,,,,
0,강남구,2018-01-01,0.0,0.0
1,강남구,2018-01-02,0.0,0.0
2,강남구,2018-01-03,0.0,0.0
3,강남구,2018-01-04,0.0,0.0
4,강남구,2018-01-05,0.0,0.0
...,...,...,...,...
19302,NaN,NaT,NaN,NaN
19303,NaN,NaT,NaN,NaN
19304,NaN,NaT,NaN,NaN


#### 두 데이터 병합

In [21]:
ta.head()

,발생일,사망자수,중상자수,경상자수,발생지_시군구
363827,2018-01-01,0,0,2,용산구
363828,2018-01-01,0,1,1,강동구
363829,2018-01-01,0,0,1,송파구
363830,2018-01-01,0,1,0,성북구
363835,2018-01-01,0,0,1,강북구


In [22]:
rainfall.head()

,발생지_시군구,발생일,강수량(mm),1시간최다강수량(mm)
index,,,,
0,강남구,2018-01-01,0.0,0.0
1,강남구,2018-01-02,0.0,0.0
2,강남구,2018-01-03,0.0,0.0
3,강남구,2018-01-04,0.0,0.0
4,강남구,2018-01-05,0.0,0.0


In [73]:
###### 두 데이터프레임 병합
merge = pd.merge(left=ta, right=rainfall, how='left',
                  on=['발생일','발생지_시군구'], sort=False)
merge

# 발생지_시군구 -> 발생지 컬럼명바꾸기
merge2 = merge.rename(columns={"발생지_시군구":"발생지"})
merge2

,발생일,사망자수,중상자수,경상자수,발생지,강수량(mm),1시간최다강수량(mm)
0,2018-01-01,0,0,2,용산구,0.0,0.0
1,2018-01-01,0,1,1,강동구,0.0,0.0
2,2018-01-01,0,0,1,송파구,0.0,0.0
3,2018-01-01,0,1,0,성북구,0.0,0.0
4,2018-01-01,0,0,1,강북구,0.0,0.0
...,...,...,...,...,...,...,...
78048,2019-12-31,0,0,0,중랑구,0.0,0.0
78049,2019-12-31,0,0,1,강남구,0.0,0.0
78050,2019-12-31,0,0,1,송파구,0.0,0.0
78051,2019-12-31,0,0,0,강남구,0.0,0.0


#### 결측치 처리

In [74]:
##### 결측치 확인
merge2.isnull().sum()
merge2[merge2.isnull().any(1)]

,발생일,사망자수,중상자수,경상자수,발생지,강수량(mm),1시간최다강수량(mm)
10,2018-01-01,0,1,2,영등포구,NaN,NaN
11,2018-01-01,0,1,0,영등포구,NaN,NaN
18,2018-01-01,0,1,1,중구,NaN,NaN
40,2018-01-01,0,1,0,영등포구,NaN,NaN
50,2018-01-01,0,1,0,영등포구,NaN,NaN
...,...,...,...,...,...,...,...
77973,2019-12-31,0,0,1,중구,NaN,NaN
78009,2019-12-31,0,0,1,영등포구,NaN,NaN
78028,2019-12-31,0,0,1,중구,NaN,NaN
78035,2019-12-31,0,0,1,중구,NaN,NaN


In [75]:
# 강수량 결측치 -> 0으로 대체
merge2 = merge2.fillna(0)
merge2.isnull().sum()

발생일             0
사망자수            0
중상자수            0
경상자수            0
발생지             0
강수량(mm)         0
1시간최다강수량(mm)    0
dtype: int64

In [76]:
##### 발생일, 발생지 별로 교통사고수 구하기
total_ta = merge2.groupby(["발생일","발생지"]).size()
total_ta
type(total_ta)
# 데이터프레임으로 변환
total_ta = pd.DataFrame(total_ta)
total_ta
# 열 이름 설정
total_ta = total_ta.rename(columns={0:"교통사고수"})
total_ta

교통사고수
발생일        발생지       
2018-01-01 강남구      4
           강동구      2
           강북구      4
           강서구      1
           관악구      5
...               ...
2019-12-31 용산구      3
           은평구      2
           종로구      2
           중구       3
           중랑구      5

[17599 rows x 1 columns]

In [77]:
##### 발생일, 발생지 별로 교통사고수는 합계를, 강수량은 평균으로 묶기
df1 = merge2.groupby(["발생일","발생지"]).sum()[["사망자수","중상자수","경상자수"]]
df2 = merge2.groupby(["발생일","발생지"]).mean()[["강수량(mm)","1시간최다강수량(mm)"]]
df3 = df1.join(df2)
df3

사망자수  중상자수  경상자수  강수량(mm)  1시간최다강수량(mm)
발생일        발생지                                         
2018-01-01 강남구     0     1     2      0.0           0.0
           강동구     0     1     2      0.0           0.0
           강북구     0     1     4      0.0           0.0
           강서구     0     0     1      0.0           0.0
           관악구     0     1     7      0.0           0.0
...              ...   ...   ...      ...           ...
2019-12-31 용산구     0     0     3      0.0           0.0
           은평구     0     0     4      0.0           0.0
           종로구     0     0     2      0.0           0.0
           중구      0     0     3      0.0           0.0
           중랑구     0     1     2      0.0           0.0

[17599 rows x 5 columns]

### Master table

In [78]:
##### total_ta와 df3 병합하기
df = df3.join(total_ta)
df

사망자수  중상자수  경상자수  강수량(mm)  1시간최다강수량(mm)  교통사고수
발생일        발생지                                                
2018-01-01 강남구     0     1     2      0.0           0.0      4
           강동구     0     1     2      0.0           0.0      2
           강북구     0     1     4      0.0           0.0      4
           강서구     0     0     1      0.0           0.0      1
           관악구     0     1     7      0.0           0.0      5
...              ...   ...   ...      ...           ...    ...
2019-12-31 용산구     0     0     3      0.0           0.0      3
           은평구     0     0     4      0.0           0.0      2
           종로구     0     0     2      0.0           0.0      2
           중구      0     0     3      0.0           0.0      3
           중랑구     0     1     2      0.0           0.0      5

[17599 rows x 6 columns]